In [11]:
import pandas as pd 
import ast
import numpy as np
import string
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import operator
import math
from tqdm import tqdm
import time
import itertools
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /Users/jeroe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jeroe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jeroe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
tf = pd.read_csv('../../../data/bm25/bm25_no_survey/merged_courses_tf.csv', header=0, index_col=0)
tf_norm = pd.read_csv('../../../data/bm25/bm25_no_survey/merged_courses_tf_norm.csv', header=0, index_col=0)
idf = pd.read_csv('../../../data/bm25/bm25_no_survey/merged_courses_idf.csv', header=0, index_col=0)
df = pd.read_csv('../../../data/bm25/bm25_no_survey/merged_courses_df.csv', header=0, index_col=0)
glove_kv = '../../../pretrained_corpus/glove_6B_300d.kv'   # pretrained vectors for query expansion
added_data = pd.read_csv('../../../data/bm25/bm25_relevance_feedback/TrainingSampleQuery.csv', header = 0 , index_col = 0)
association_matrix = pd.read_csv('../../../data/association_matrix/association_matrix_trg.csv', header = 0, index_col = 0)
norm_association_matrix = pd.read_csv('../../../data/association_matrix/norm_association_matrix_trg.csv', header = 0, index_col = 0)

In [14]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def process_query(query):
    query = query.translate(str.maketrans('', '', string.punctuation))
    query = str(np.char.lower(query))
    query = ''.join([i for i in query if not i.isdigit()])
    query = remove_stopwords(query)
    query = [lemmatizer.lemmatize(w, get_wordnet_pos(w))
             for w in nltk.word_tokenize(query)]
    query = ' '.join([str(elem) for elem in query])
    query = query.translate(str.maketrans('', '', string.punctuation))
    return query.split()

def expand_query(query,glove_kv,topn):
    model = KeyedVectors.load(glove_kv)
    expanded_query = []
    for word in query:
        expanded_query.append(word)
        if word in model:
            for close_word in model.most_similar(word, topn = topn):
                expanded_query.append(close_word[0])
    return expanded_query


In [15]:
def get_associated_words(query, matrix):
    '''
    for each word in original query, add on a word which is best associated with it via the corpus
    '''
    extended_query = query.copy()
    for word in query:
        if word in matrix.index:
            new_word = matrix[word].astype(float).idxmax()
            extended_query.append(new_word)
    return extended_query

def get_top_k_associated_words(relevant_courses, tf, matrix, k):
    '''
    get top k associated words and reformulate back into the query
    '''
    total_new_words = []
    for course in relevant_courses:
        words = tf.index[tf[course]>0].tolist()
        # filtered_matrix = matrix[matrix.index.isin(words)]
        associated_words = {}
        for word in words:
            associated_words[matrix[word].astype(float).idxmax()] = matrix[word].astype(float).max()
        new_words = sorted(associated_words, key=associated_words.get, reverse=True )[:k]
        total_new_words += new_words
    return total_new_words

def create_association_matrix(tf):
    '''
    takes in tf dataframe to get association matrix df
    '''
    unique_words = tf.index.values.tolist()
    ls = list(itertools.combinations(tf.index, 2))
    for idx in tf.index:
        ls.append((idx,idx))
    associations = {}
    for item in ls:
        k = item[0]+item[1]
        associations[k] = sum(tf.loc[item[0]]*tf.loc[item[1]])
    association_matrix = pd.DataFrame(0,index=unique_words,columns=unique_words)
    for i in tqdm(unique_words):
        for j in unique_words:
            # association_matrix.loc[i,j] = sum(tf.loc[i]*tf.loc[j])
            if i+j in associations:
                association_matrix.loc[i,j] = associations[i+j]
            elif j+i in associations:
                association_matrix.loc[i,j] = associations[j+i]
    return association_matrix,unique_words

def create_norm_association_matrix(association_matrix, unique_words):
    norm_association_matrix = pd.DataFrame(index=unique_words,columns=unique_words)
    for i in tqdm(unique_words):
        for j in unique_words:
            if i!=j:
                norm_association_matrix.loc[i,j] = association_matrix.loc[i,j]/(association_matrix.loc[i,i]+association_matrix.loc[j,j]-association_matrix.loc[i,j])
    return norm_association_matrix

In [16]:
def update_scores(tf, relevant_courses, associated_words):
    courses = tf.columns.tolist()
    for course in relevant_courses:
        for word in associated_words:
            if word in tf.index.tolist():
                tf[course][word] += 1
            else:
                idx = courses.index(course)
                temp_lst = [1 if i == idx else 0 for i in range(len(courses))]
                temp_df = pd.DataFrame([temp_lst], columns=courses, index=[word])
                tf = tf.append(temp_df)
    tf.sort_index()

    tf_norm = tf.apply(lambda x: x/x.max(), axis=0) # normalise each column by dividing the max frequency
    tf_norm = tf_norm.replace(np.nan, 0)

    df_arr = np.count_nonzero(tf, axis=1) # compute df: count the nunber of non zero columns in each role 
    df = pd.DataFrame(data={'df':df_arr}, index = tf.index.tolist())

    idf = df.copy()
    idf['df'] = idf['df'].apply(lambda freq: math.log10((len(tf.columns.tolist())) / (freq))) # calc idf
    idf = idf.rename({'df': 'idf'}, axis=1, inplace=False)
    idf = idf.replace(np.nan, 0)
    
    return tf, tf_norm, df, idf

In [17]:
# bm-25 processing 
def bm25_basic(query, doc, tf, tf_norm, idf, vocab, avg_doc_len, k1=1.5, b=0.75):
    ''''
    returns the score for a document given a query based on the basic bm25 algorithm
    '''
    score = 0.0
    for term in query:
        if term not in vocab:
            continue
        numerator = idf['idf'][term] * tf_norm[doc][term] * (k1 + 1)
        doc_len = tf[doc].to_numpy().sum()
        denominator = tf_norm[doc][term] + k1 * \
            (1 - b + b * doc_len / avg_doc_len)
        score += (numerator / denominator)
    return score

def bm25_reformulated(query, doc, relevant_courses, tf, df, vocab, tf_norm, avg_doc_len, k1=1.5, b=0.75, k3=1.5):
    score = 0.0
    vr = len(relevant_courses)  # total retrieved relevant docs
    for term in query:
        if term not in vocab:
            continue
        vr_t = 0  # total retrieved relevant docs where term t appears
        for course in relevant_courses:
            words = tf.index[tf[course]>0].tolist()
            if term in words:
                vr_t+=1
        vnr_t = vr-vr_t  # total retrieved relevant docs where term t does not appear
        df_t = float(df.loc[term]) # document frequency with given term 
        doc_len = tf[doc].to_numpy().sum()
        tf_d = float(tf_norm[doc][term]) + 0.0001 # term frequency in document (include 0.0001 smoothing if not code will break lol)
        tf_q = query.count(term)/len(query) # term frequency in reformulated query normalised
        N = len(tf.columns) # total number of documents
        part_a = ((abs(vr_t)+0.5)/(abs(vnr_t)+0.5))/((df_t-abs(vr_t)+0.5)/(N-df_t-abs(vr)+abs(vr_t)+0.5))
        part_b = ((k1+1)*tf_d)/((k1*((1-b)+b*(doc_len/avg_doc_len)))+tf_d)
        part_c = ((k3+1)*tf_q)/(k3+tf_q)

        score += math.log10(part_a*part_b*part_c)
    return score

def bm25_prediction(query, tf, tf_norm, df, idf, vocab, avg_doc_len, reformulated, relevant_courses=[]):
    '''
    ranks the documents based on the scores of all the documents
    reformulated: if true run bm25_reformulated algorithm, else run bm25_basic algorithm.
    '''
    courses = tf.columns.tolist()
    result = {}
    if reformulated is False:
        print(f'running basic bm25')
        for course in courses:
            result[course] = bm25_basic(query, course, tf, tf_norm, idf, vocab, avg_doc_len)
    elif reformulated is True:
        print(f'running bm25 for reformulated query')
        for course in courses:
            result[course] = bm25_reformulated(query, course, relevant_courses, tf, df, vocab, tf_norm, avg_doc_len)      
    sorted_result = dict(
        sorted(result.items(), key=operator.itemgetter(1), reverse=True))
    ls = []
    for k, v in sorted_result.items():
        ls.append(k)
        # print(f"{k}: {v}")
    return result, ls



In [18]:
# cleaning of survey results used for training

for idx_i, courses in enumerate(added_data['expectedElectivesInOrder']):
    added_data['expectedElectivesInOrder'][idx_i] = ast.literal_eval(courses)
    for idx_j,course in enumerate(added_data['expectedElectivesInOrder'][idx_i]):
        if course == " 50.035 Computer Vision":
            added_data['expectedElectivesInOrder'][idx_i][idx_j] = '50.035 Computer Vision'
        elif course == "50.043 Database Systems / Database and Big Data Systems (for class 2021)":
            added_data['expectedElectivesInOrder'][idx_i][idx_j] = '50.043 Database Systems'
        elif course == "40.302 Advanced Optim/ 40.305 Advanced Stochastic":
            added_data['expectedElectivesInOrder'][idx_i].remove("40.302 Advanced Optim/ 40.305 Advanced Stochastic")
            added_data['expectedElectivesInOrder'][idx_i].insert(idx_j,'40.302 Advanced Topics in Optimisation#')
            # added_data['expectedElectivesInOrder'][idx_i].insert(idx_j+1, '40.305 Advanced Topics in Stochastic Modelling#')


/Users/jeroe/opt/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
added_data.head(3)

,querySample,expectedElectivesInOrder,expectedElectivesInOrderSumWordCount,queryType
0,"computational, analysis, solidity, mongodb, ev...","[50.038 Computational Data Science, 50.033 Fou...","[2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",1
1,"analysis, ampl, cast, sklearn, ui",[40.321 Airport Systems Modelling and Simulati...,"[6.0, 3.0, 2.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",1
2,"real, soup, social, computational, tensorflow","[50.035 Computer Vision, 50.038 Computational ...","[4.0, 3.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1


In [20]:
# entire pipeline for training phase
top_retrieved = 10
start_time = time.time()

for idx,row in added_data.iterrows(): # accessing all sample queries from training set
    print(f'Training iteration: {idx+1}')
    vocab = tf.index.tolist()  # unique words
    total_length = tf.to_numpy().sum()
    avg_doc_len = total_length / len(tf.columns) # average document length across all courses
    query = row['querySample']  # take in query from training sample
    print(f'original query from training sample: {query}')
    query = process_query(query) # lemitize query 
    query = get_associated_words(query, norm_association_matrix) # included associated terms
    print(f'query expansion with correlation matrix: {query}')
    query = expand_query(query,glove_kv,topn=3)  # expand query by including words from pretrianed w2v corpus
    print(f'query expansion with pretrained corpus: {query}')
    result, ls = bm25_prediction(query=query, df=df, tf=tf, tf_norm=tf_norm, idf=idf, vocab=vocab, avg_doc_len=avg_doc_len, reformulated=False)  # intial bm25
    predicted = ls[:top_retrieved] # retrieve top 10 courses from predictions
    print(f'initial predicted courses: ')
    for course in predicted: 
        print(course)
    print('')
    gold_standard = added_data['expectedElectivesInOrder'][idx]
    relevant_courses = []
    for i in range(1,top_retrieved+1):
        if gold_standard[i-1] == predicted[i-1]:
            relevant_courses.append(predicted[i-1])
            print(f'Relevant & Retrieved: Rank {i} {predicted[i-1]}') # Documents which are relevant are retrieved, comparing with gold standard training sample
    associated_words = get_top_k_associated_words(relevant_courses, tf, norm_association_matrix, k=3)  # get top 3 associated words from each relevant and retrieved course 
    query+=associated_words  # add associated words to orginal query to form the eformulated query
    print('')
    print(f'reformulated query:{query}')
    result, ls = bm25_prediction(query=query, df=df, tf=tf, tf_norm=tf_norm, idf=idf, vocab=vocab, avg_doc_len=avg_doc_len, relevant_courses=relevant_courses, reformulated=True) # bm25 reformulated
    predicted_reformulated = ls[:top_retrieved] # retrieve top 10 courses from predictions
    print(f'prediction after query reformulation')
    for course in predicted_reformulated:
        print(course)
    
    tf, tf_norm, df, idf = update_scores(tf=tf, relevant_courses=relevant_courses, associated_words=associated_words)  # update tf, tf_norm, idf, df
    association_matrix,unique_words = create_association_matrix(tf=tf)
    norm_association_matrix = create_norm_association_matrix(association_matrix = association_matrix,unique_words = unique_words) # update correlation matrix
    print('updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...')
    print(f'time elapsed: {(time.time()-start_time)//60}min {(time.time()-start_time)%60}s')
    print('')



print(f'time elapsed: {(time.time()-start_time)//60}min {(time.time()-start_time)%60}s') 
# saving trained scores 
tf.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_tf_trained.csv')
tf_norm.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_tf_norm_trained.csv')
df.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_df_trained.csv')
idf.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_idf_trained.csv')
association_matrix.to_csv('../../../data/bm25/bm25_relevance_feedback/association_matrix_trained.csv')
norm_association_matrix.to_csv('../../../data/bm25/bm25_relevance_feedback/association_matrix_norm_trained.csv')


Training iteration: 1
original query from training sample: computational, analysis, solidity, mongodb, evaluation
query expansion with correlation matrix: ['computational', 'analysis', 'solidity', 'mongodb', 'evaluation', 'fabrication', 'framework', 'set']
query expansion with pretrained corpus: ['computational', 'computation', 'mathematical', 'algorithms', 'analysis', 'analyses', 'study', 'data', 'solidity', 'soundness', 'robustness', 'predictability', 'mongodb', 'nosql', 'scriptable', 'rhizaria', 'evaluation', 'assessment', 'evaluations', 'evaluate', 'fabrication', 'wafer', 'fabricated', 'fabrications', 'framework', 'implementation', 'frameworks', 'principles', 'set', 'setting', 'sets', 'up']
running basic bm25
initial predicted courses: 
50.048 Computational Fabrication
40.323 Equity Valuation
40.230 Sustainable Engineering
50.038 Computational Data Science
01.117 Brain-Inspired Computing and its Applications (Term 8)
50.006 User Interface Design and Implementation
40.324 Fundamenta

100%|██████████| 1396/1396 [03:40<00:00,  6.34it/s]


updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...
time elapsed: 11.0min 24.224261045455933s

Training iteration: 2
original query from training sample: analysis, ampl, cast, sklearn, ui
query expansion with correlation matrix: ['analysis', 'ampl', 'cast', 'sklearn', 'ui', 'framework', 'airtop', 'automatic']
query expansion with pretrained corpus: ['analysis', 'analyses', 'study', 'data', 'ampl', 'mangxamba', 'ufdots', 'uninitialized', 'cast', 'casting', 'casts', 'actors', 'sklearn', 'ui', 'fiachrach', 'uí', 'pak', 'framework', 'implementation', 'frameworks', 'principles', 'airtop', 'automatic', 'machine', 'automatically', 'automated']
running basic bm25
initial predicted courses: 
40.323 Equity Valuation
50.038 Computational Data Science
01.116 AI for Healthcare (Term 7)
40.321 Airport Systems Modelling and Simulation
50.007 Machine Learning
50.043 Database Systems
40.319 Statistical and Machine Learning
50.045 Information Retrieval
40.316 Game Theory
0

100%|██████████| 1396/1396 [03:33<00:00,  6.54it/s]


updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...
time elapsed: 22.0min 19.784332752227783s

Training iteration: 3
original query from training sample: real, soup, social, computational, tensorflow
query expansion with correlation matrix: ['real', 'soup', 'social', 'computational', 'tensorflow', 'world', 'fairness', 'fabrication', 'localization']
query expansion with pretrained corpus: ['real', 'true', 'just', 'what', 'soup', 'soups', 'stew', 'noodle', 'social', 'welfare', 'education', 'political', 'computational', 'computation', 'mathematical', 'algorithms', 'tensorflow', 'world', 'ever', 'time', 'global', 'fairness', 'honesty', 'impartiality', 'objectivity', 'fabrication', 'wafer', 'fabricated', 'fabrications', 'localization', 'localisation', 'subcellular', 'internationalization']
running basic bm25
initial predicted courses: 
50.048 Computational Fabrication
01.104 Networked Life
50.017 Graphics and Visualisation
50.035 Computer Vision
40.319 Statist

...

updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...
time elapsed: 731.0min 39.64076900482178s

Training iteration: 29
original query from training sample: ampl, python, optimize, aws, learn
query expansion with correlation matrix: ['ampl', 'python', 'optimize', 'aws', 'learn', 'abstract', 'arp', 'differ', 'algorithm']
query expansion with pretrained corpus: ['ampl', 'mangxamba', 'ufdots', 'uninitialized', 'python', 'monty', 'perl', 'cleese', 'optimize', 'optimizing', 'optimise', 'readjust', 'aws', 'khazraj', 'rds', 'nahas', 'learn', 'learned', 'teach', 'understand', 'abstract', 'expressionist', 'expressionism', 'figurative', 'arp', 'schnitger', '2600', 'synthesiser', 'differ', 'differing', 'differed', 'vary', 'algorithm', 'algorithms', 'iterative', 'approximation']
running basic bm25
initial predicted courses: 
40.317 Financial Systems Design
50.007 Machine Learning
50.017 Graphics and Visualisation
40.319 Statistical and Machine Learning
50.043 Database 

100%|██████████| 1396/1396 [03:37<00:00,  6.42it/s]


updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...
time elapsed: 742.0min 45.90479397773743s

Training iteration: 30
original query from training sample: security, solidity, c, statistic, calculation
query expansion with correlation matrix: ['security', 'solidity', 'c', 'statistic', 'calculation', 'common']
query expansion with pretrained corpus: ['security', 'forces', 'military', 'intelligence', 'solidity', 'soundness', 'robustness', 'predictability', 'c', 'b', 'f', 'g', 'statistic', 'statistics', 'bps', 'statistical', 'calculation', 'calculations', 'calculating', 'calculate', 'common', 'commonly', 'example', 'similar']
running basic bm25
initial predicted courses: 
50.020 Network Security
01.117 Brain-Inspired Computing and its Applications (Term 8)
50.044 System Security
50.042 Foundations of Cybersecurity
40.305 Advanced Topics in Stochastic Modelling#
40.232 Water Resources Management
40.319 Statistical and Machine Learning
50.021 Artificial Intelli

100%|██████████| 1396/1396 [03:40<00:00,  6.33it/s]


updated tf, tf_norm, df, idf, asociation matrix, association matrix norm scores...
time elapsed: 754.0min 18.709813117980957s

time elapsed: 754.0min 18.714138984680176s


In [41]:
# main code, once pipeline is up, will transfer all to training.py
start_time = time.time()
vocab = tf.index.tolist()
total_length = tf.to_numpy().sum()
avg_doc_len = total_length / len(tf.columns)
top_retrieved = 10
sample = 0

# take for example first iteration 
query = added_data['querySample'][sample]
# lemitization
query = process_query(query)
# included associated terms
query = get_associated_words(query, norm_association_matrix)
# include words from pretrained w2v corpus
query = expand_query(query,glove_kv,topn=3)
print('initial query after passing through association matrix and expanded with pretrained corpus')
print(query)
result, ls = bm25_prediction(query=query, df=df, tf=tf, tf_norm=tf_norm, idf=idf, vocab=vocab, \
                             avg_doc_len=avg_doc_len, reformulated=False)
# only want top 10 predicted courses
predicted = ls[:top_retrieved]
print(f'initial prediction: ')
for course in predicted:
    print(course)
print('')

gold_standard = added_data['expectedElectivesInOrder'][sample]
relevant_courses = []

# print out documents which are relevant are retrieved, comparing with gold standard
for i in range(1,top_retrieved+1):
    if gold_standard[i-1] == predicted[i-1]:
        relevant_courses.append(predicted[i-1])
        print(f'Relevant & Retrieved: Rank {i} {predicted[i-1]}')

# get top 3 associated words from each relevant and retrieved course 
associated_words = get_top_k_associated_words(relevant_courses, tf, norm_association_matrix, k=3)
# add associated words to orginal query to form reformulated query
query+=associated_words
print('')
print('reformulated query after passing through association matrix')
print(query)

# insert reformulated query into bm25_reformulated 
result, ls = bm25_prediction(query=query, df=df, tf=tf, tf_norm=tf_norm, idf=idf, vocab=vocab, \
                             avg_doc_len=avg_doc_len, relevant_courses=relevant_courses, reformulated=True)
predicted_reformulated = ls[:top_retrieved]
print(f'prediction after query reformulation')
for course in predicted_reformulated:
    print(course)

# update tf, tf_norm, idf, df
tf, tf_norm, df, idf = update_scores(tf=tf, relevant_courses=relevant_courses, associated_words=associated_words)

# update correlation matrix
association_matrix,unique_words = create_association_matrix(tf=tf)
norm_association_matrix = create_norm_association_matrix(association_matrix = association_matrix,unique_words = unique_words)



print(f'time elapsed: {(time.time()-start_time)//60}min {(time.time()-start_time)%60}s')
tf.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_tf_trained.csv')
tf_norm.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_tf_norm_trained.csv')
df.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_df_trained.csv')
idf.to_csv('../../../data/bm25/bm25_relevance_feedback/merged_courses_idf_trained.csv')
association_matrix.to_csv('../../../data/bm25/bm25_relevance_feedback/association_matrix_trained.csv')
norm_association_matrix.to_csv('../../../data/bm25/bm25_relevance_feedback/association_matrix_norm_trained.csv')


initial query after passing through association matrix and expanded with pretrained corpus
['golang', 'cod', 'haddock', 'halibut', 'nantucket', 'feedback', 'input', 'responses', 'interaction', 'aviation', 'aircraft', 'airline', 'aerospace', 'statistic', 'statistics', 'bps', 'statistical', 'load', 'loads', 'loading', 'loaded', 'fairness', 'honesty', 'impartiality', 'objectivity']
running basic bm25
initial prediction: 
01.104 Networked Life
40.317 Financial Systems Design
50.006 User Interface Design and Implementation
40.232 Water Resources Management
50.039 Theory and Practice of Deep Learning
40.320 Airport Systems Planning and Design
01.107 Urban Transportation
01.117 Brain-Inspired Computing and its Applications (Term 8)
40.321 Airport Systems Modelling and Simulation
40.319 Statistical and Machine Learning

Relevant & Retrieved: Rank 6 40.320 Airport Systems Planning and Design

reformulated query after passing through association matrix
['golang', 'cod', 'haddock', 'halibut', 'na

  0%|          | 0/1396 [00:00<?, ?it/s]

prediction after query reformulation
40.320 Airport Systems Planning and Design
50.047 Mobile Robotics
01.104 Networked Life
50.039 Theory and Practice of Deep Learning
01.107 Urban Transportation
40.232 Water Resources Management
40.317 Financial Systems Design
01.117 Brain-Inspired Computing and its Applications (Term 8)
40.305 Advanced Topics in Stochastic Modelling#
50.006 User Interface Design and Implementation


100%|██████████| 1396/1396 [04:14<00:00,  5.49it/s]


time elapsed: 19.0min 39.429699659347534s


In [82]:
norm_asc_matrix = pd.read_csv('../../../data/bm25/bm25_relevance_feedback/association_matrix_norm_trained.csv',header = 0 , index_col = 0)
norm_asc_matrix.head()

,ability,able,abstract,abstraction,accommodate,accompanies,account,accounting,achieve,acquaint,...,window,wireless,word,wordvec,work,world,write,xl,year,zigbee
ability,NaN,0.006944,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.2,0.0,...,0.0,0.0,0.000000,0.000000,0.047619,0.000000,0.1,0.0,0.000000,0.0
able,0.006944,NaN,0.0,0.0,0.0,0.000000,0.014286,0.028369,0.0,0.0,...,0.0,0.0,0.014184,0.014286,0.060000,0.154321,0.0,0.0,0.014286,0.0
abstract,0.000000,0.000000,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
abstraction,0.000000,0.000000,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
accommodate,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [90]:
norm_asc_matrix['cod'].astype(float).idxmax()

'load'